<a href="https://colab.research.google.com/github/kutaydemiralay/MPC-Style-SCP-for-Quadrotor-Obstacle-Avoidance/blob/main/KutayDemiralay_Quadrotor_Obstacle_Avoidance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import cvxpy as cp
import numpy as np
import sympy as sp
import scipy
import scipy.integrate as itg
from sympy import MatrixSymbol, Matrix, lambdify
import plotly.graph_objects as go


def skew_symmetric_matrix_quat(w):
    w0, w1, w2 = w
    return np.array([
        [0, -w0, -w1, -w2],
        [w0, 0, -w2, w1],
        [w1, w2, 0, -w0],
        [w2, -w1, w0, 0]
    ])

def skew_symmetric_matrix(w):
    w0, w1, w2 = w
    return np.array([
        [0, -w2, w1],
        [w2, 0, -w0],
        [-w1, w0, 0]
    ])

def quat_to_dcm(q):
    q_norm = sp.sqrt(q[0]**2 + q[1]**2 + q[2]**2 + q[3]**2)
    w, x, y, z = q / q_norm
    return np.array([[1 - 2 * (y ** 2 + z ** 2), 2 * (x * y - z * w), 2 * (x * z + y * w)],
                     [2 * (x * y + z * w), 1 - 2 * (x ** 2 + z ** 2), 2 * (y * z - x * w)],
                     [2 * (x * z - y * w), 2 * (y * z + x * w), 1 - 2 * (x ** 2 + y ** 2)]])

def augmented_dynamics(t, state, control_current, control_slope, t_start, params, prop, obstacles=None):
    if not prop:
        control = control_current + control_slope * (t - t_start)
    else:
        control = control_current

    r_i = state[:3]
    v_i = state[3:6]
    q_bi = state[6:10]
    w_b = state[10:13]

    u_mB = control[:3]
    tau_i = control[3:]

    q_norm = np.linalg.norm(q_bi)
    q_bi = q_bi / q_norm

    p_i_dot = v_i
    v_i_dot = (1 / params['m']) * quat_to_dcm(q_bi) @ (u_mB) + np.array([0, 0, params['g']])
    q_bi_dot = 0.5 * skew_symmetric_matrix_quat(w_b) @ q_bi
    w_b_dot = np.diag(1 / params['J_b']) @ (tau_i - skew_symmetric_matrix(w_b) @ np.diag(params['J_b']) @ w_b)

    state_dot = np.hstack([p_i_dot, v_i_dot, q_bi_dot, w_b_dot])

    if obstacles is not None:
        g_dot = np.zeros(len(obstacles))
        augmented_state_dot = np.hstack([state_dot, g_dot])
        return augmented_state_dot
    else:
        return state_dot

def analytical_jacobians(params: dict):
    r = MatrixSymbol('r', 3, 1)
    v = MatrixSymbol('v', 3, 1)
    q = MatrixSymbol('q', 4, 1)
    w = MatrixSymbol('w', 3, 1)
    f = MatrixSymbol('f', 3, 1)
    tau = MatrixSymbol('tau', 3, 1)

    r_dot = v
    v_dot = (1 / params['m']) * quat_to_dcm(q) @ f + Matrix([0, 0, params['g']])
    q_dot = 0.5 * skew_symmetric_matrix_quat(w) @ q
    w_dot = np.diag(1 / params['J_b']) @ (tau - skew_symmetric_matrix(w) @ np.diag(params['J_b']) @ w)

    state_dot = Matrix.vstack(Matrix(r_dot), Matrix(v_dot), Matrix(q_dot), Matrix(w_dot))

    A_expr = state_dot.jacobian(Matrix([r, v, q, w]))
    B_expr = state_dot.jacobian(Matrix([f, tau]))

    A = lambdify((r, v, q, w, f, tau), A_expr, "numpy")
    B = lambdify((r, v, q, w, f, tau), B_expr, "numpy")

    return A, B

def derivative(t, V, control_current, control_next, A, B, obstacles, params):
    n_x = params['n_states']
    n_u = params['n_controls']

    i0 = 0
    i1 = n_x
    i2 = i1 + n_x * n_x
    i3 = i2 + n_x * n_u
    i4 = i3 + n_x * n_u
    i5 = i4 + n_x

    x = V[i0:i1]

    beta = t / params['dt_ss']
    alpha = 1 - beta

    u = control_current + beta * (control_next - control_current)

    A_aug = np.zeros((n_x, n_x))
    B_aug = np.zeros((n_x, n_u))

    p, v, q, w, f, tau = np.expand_dims(x[:3], 1), np.expand_dims(x[3:6], 1), np.expand_dims(x[6:10], 1), np.expand_dims(x[10:params['n_states'] - params['n_obs']], 1), np.expand_dims(u[:3], 1), np.expand_dims(u[3:], 1)

    A_subs = A(p, v, q, w, f, tau)
    A_aug[:params['n_states'] - params['n_obs'], :params['n_states'] - params['n_obs']] = A_subs

    B_subs = B(p, v, q, w, f, tau)
    B_aug[:params['n_states'] - params['n_obs']] = B_subs

    f_subs = augmented_dynamics(t, x, u, None, None, params, True, obstacles)

    z_t = np.squeeze(f_subs) - np.matmul(A_aug, x) - np.matmul(B_aug, u)

    derivative = np.zeros_like(V)
    derivative[i0:i1] = f_subs.T
    derivative[i1:i2] = np.matmul(A_aug, V[i1:i2].reshape((n_x, n_x))).reshape(-1)
    derivative[i2:i3] = (np.matmul(A_aug, V[i2:i3].reshape((n_x, n_u))) + B_aug * alpha).reshape(-1)
    derivative[i3:i4] = (np.matmul(A_aug, V[i3:i4].reshape((n_x, n_u))) + B_aug * beta).reshape(-1)
    derivative[i4:i5] = (np.matmul(A_aug, V[i4:i5]).reshape(-1) + z_t)
    return derivative

def calculate_discretization(x, u, A, B, obstacles, params):
    n_x = params['n_states']
    n_u = params['n_controls']

    i0 = 0
    i1 = n_x
    i2 = i1 + n_x * n_x
    i3 = i2 + n_x * n_u
    i4 = i3 + n_x * n_u
    i5 = i4 + n_x

    V0 = np.zeros(i5)
    V0[i1:i2] = np.eye(n_x).reshape(-1)

    f_bar = np.zeros((n_x, params['n'] - 1))
    A_bar = np.zeros((n_x * n_x, params['n'] - 1))
    B_bar = np.zeros((n_x * n_u, params['n'] - 1))
    B_minus_bar = np.zeros((n_x * n_u, params['n'] - 1))
    z_bar = np.zeros((n_x, params['n'] - 1))

    for k in range(params['n'] - 1):
        V0[i0:i1] = x[:, k]

        V = itg.solve_ivp(derivative, (0, params['dt_ss']), V0, args=(u[:, k], u[:, k + 1], A, B, obstacles, params), method='DOP853').y[:, -1]

        f_bar[:, k] = V[i0:i1]
        Phi = V[i1:i2].reshape((n_x, n_x))
        A_bar[:, k] = Phi.flatten(order='F')
        B_bar[:, k] = (V[i2:i3].reshape((n_x, n_u))).flatten(order='F')
        B_minus_bar[:, k] = (V[i3:i4].reshape((n_x, n_u))).flatten(order='F')
        z_bar[:, k] = (V[i4:i5])

    return A_bar, B_bar, B_minus_bar, z_bar

def simulate_nonlinear_with_wind(initial_state, controls, params, dt, wind_vector):
    states = [initial_state]

    t_eval_opt = np.arange(0, params['total_time'] - params['dt_ss'], params['dt_ss'])
    for i, t in enumerate(t_eval_opt):
        controls_current = controls[:, i]

        controls_next = controls[:, i + 1]
        controls_slope = (controls_next - controls_current) / params['dt_ss']
        t_eval = np.arange(t, t + params['dt_ss'] + 1e-10, params['dt_sim'])

        sol = itg.solve_ivp(augmented_dynamics, (t, t + params['dt_ss'] + 1e-10), states[-1], args=(controls_current, controls_slope, t, params, False, None), t_eval=t_eval, rtol=1e-6, atol=1e-6)
        for k in range(1, sol.y.shape[1]):
            wind_perturbed_state = sol.y[:, k].copy()
            wind_perturbed_state[:3] += wind_vector  # Add wind vector here
            states.append(wind_perturbed_state)
    return np.array(states)

def g_bar_obs(obs, p_traj):
    result = []
    for p in p_traj.T:
        result.append(1 - (np.linalg.norm(p - obs['center'])**2) / (obs['radius'] ** 2))
    return np.array(result)

def grad_g_bar_obs(obs, p_traj):
    result = []
    for p in p_traj.T:
        result.append(-4 * np.maximum(0, g_bar_obs(obs, np.expand_dims(p, 1))) * (p - obs['center']) / (obs['radius'] ** 2))
    return np.array(result).T

def parameters():
    # Weight for the tracking cost in the optimization problem
    w_tr = 1E2

    # Weight for obstacle avoidance in the optimization problem
    w_o = 20

    # Weight for fuel consumption minimization in the optimization problem
    w_fuel = 1

    # Weight for velocity constraint violations in the optimization problem
    w_vc = 1E2

    # Scaling factor to normalize the weights
    scale = max(w_tr, w_o, w_fuel, w_vc)

    # Number of obstacles in the environment
    n_obs = 3

    # Number of state variables (13 for the quadrotor + 1 for each obstacle)
    n_states = 13 + n_obs

    # Number of control inputs (6 for the quadrotor)
    n_controls = 6

    # Initial state of the quadrotor [position (3), velocity (3), quaternion (4), angular velocity (3)]
    initial_state = np.array([-10, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0])

    # Final desired state of the quadrotor
    final_state = np.array([10, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0])

    # Initial control input (thrust and torques) for the quadrotor
    initial_control = np.array([0, 0, 9.81, 0, 0, 0])

    # Maximum allowable state values (e.g., position, velocity, etc.)
    max_state = np.array([10, 10, 10, 50, 50, 50, 1, 1, 1, 1, 10, 10, 10])
    for i in range(n_obs):
        max_state = np.hstack([max_state, [1]])  # Adding limits for obstacle-related states

    # Minimum allowable state values
    min_state = np.array([-10, -10, -10, -50, -50, -50, -1, -1, -1, -1, -10, -10, -10])
    for i in range(n_obs):
        min_state = np.hstack([min_state, [0]])  # Adding limits for obstacle-related states

    # Maximum allowable control inputs (thrust and torques)
    max_control = np.array([0, 0, 250, 20, 20, 2])

    # Minimum allowable control inputs
    min_control = np.array([0, 0, 0, -10, -10, -1])

    # Centers of the obstacles in the environment
    obstacle_centers = [
        np.array([-5, -1.1, 0]),  # Center of the first obstacle
        np.array([0, 0, 0]),      # Center of the second obstacle
        np.array([5, 0.9, 0])     # Center of the third obstacle
    ]

    # Radii of the obstacles (assuming spherical obstacles)
    obstacle_radii = [1.7, 1.2, 1.8]  # Predefined radii for the spherical obstacles

    # State scaling matrix for normalization (based on state limits)
    S_x = np.diag(np.maximum(np.ones(n_states), abs(min_state - max_state) / 2))

    # State offset vector for normalization
    c_x = (max_state + min_state) / 2

    # Control scaling matrix for normalization (based on control limits)
    S_u = np.diag(np.maximum(np.ones(n_controls), abs(min_control - max_control) / 2))

    # Control offset vector for normalization
    c_u = (max_control + min_control) / 2

    # Total time duration of the trajectory (seconds)
    total_time = 8

    # Time step for state-space discretization (seconds)
    dt_ss = 0.4

    # Simulation time step for numerical integration (seconds)
    dt_sim = 0.025

    # Dictionary containing all parameters
    return dict(
        g=-9.81,  # Gravitational acceleration (m/s^2)
        m=1.0,  # Mass of the quadrotor (kg)
        J_b=np.array([1, 1, 1]),  # Inertia matrix of the quadrotor (assumed diagonal for simplicity)
        total_time=total_time,  # Total simulation time (s)
        dt_ss=dt_ss,  # Discretization time step (s)
        dt_sim=dt_sim,  # Simulation time step (s)
        k_max=20,  # Maximum number of iterations for the optimization algorithm
        n=int(total_time / dt_ss),  # Number of time steps in the trajectory
        n_states=n_states,  # Number of state variables
        n_controls=n_controls,  # Number of control inputs
        n_obs=n_obs,  # Number of obstacles
        obstacle_centers=obstacle_centers,  # Centers of obstacles
        obstacle_radii=obstacle_radii,  # Radii of obstacles
        S_x=S_x,  # State scaling matrix
        S_u=S_u,  # Control scaling matrix
        c_x=c_x,  # State offset vector
        c_u=c_u,  # Control offset vector
        w_tr=w_tr / scale,  # Normalized tracking weight
        w_o=w_o / scale,  # Normalized obstacle avoidance weight
        w_fuel=w_fuel / scale,  # Normalized fuel consumption weight
        w_vc=w_vc / scale,  # Normalized velocity constraint weight
        ep_tr=1E-3,  # Threshold for tracking error convergence
        ep_vb=1E-3,  # Threshold for boundary condition error convergence
        ep_vc=1E-7,  # Threshold for velocity constraint violation convergence
        initial_state=initial_state,  # Initial state of the quadrotor
        initial_control=initial_control,  # Initial control input
        final_state=final_state,  # Final desired state of the quadrotor
        max_state=max_state,  # Maximum state limits
        min_state=min_state,  # Minimum state limits
        max_control=max_control,  # Maximum control limits
        min_control=min_control  # Minimum control limits
    )




def controller(params):
    w_tr = cp.Parameter(nonneg=True, name='w_tr')
    w_terminal = cp.Parameter(nonneg=True, name='w_terminal')  # Define terminal cost weight parameter

    x = cp.Variable((params['n_states'], params['n']), name='x')
    dx = cp.Variable((params['n_states'], params['n']), name='dx')
    x_bar = cp.Parameter((params['n_states'], params['n']), name='x_bar')

    S_x = params['S_x']
    c_x = params['c_x']

    u = cp.Variable((params['n_controls'], params['n']), name='u')
    du = cp.Variable((params['n_controls'], params['n']), name='du')
    u_bar = cp.Parameter((params['n_controls'], params['n']), name='u_bar')

    S_u = params['S_u']
    c_u = params['c_u']

    g_bar_obs = []
    grad_g_bar_obs = []
    for k in range(params['n_obs']):
        g_bar_obs.append(cp.Parameter(params['n'] - 1, name='g_bar_obs_' + str(k)))
        grad_g_bar_obs.append(cp.Parameter((3, params['n'] - 1), name='grad_g_bar_obs_' + str(k)))

    A_prop = cp.Parameter(((params['n_states']) * (params['n_states']), params['n'] - 1), name='A_d')
    B_prop = cp.Parameter((params['n_states'] * params['n_controls'], params['n'] - 1), name='B_d')
    C_prop = cp.Parameter((params['n_states'] * params['n_controls'], params['n'] - 1), name='B_minus_d')
    z_prop = cp.Parameter((params['n_states'], params['n'] - 1), name='z_d')
    nu = cp.Variable((params['n_states'], params['n'] - 1), name='nu')

    x_unscaled = []
    u_unscaled = []
    for k in range(params['n']):
        x_unscaled.append(S_x @ x[:, k] + c_x)
        u_unscaled.append(S_u @ u[:, k] + c_u)

    cost = 0
    constr = [x_unscaled[0][:params['n_states'] - params['n_obs']] == params['initial_state'],
              x_unscaled[-1][:params['n_states'] - params['n_obs']] == params['final_state']]
    constr += [0 == x_unscaled[0] - x_bar[:, 0] - dx[:, 0]]
    constr += [0 == u_unscaled[-1] - u_bar[:, -1] - du[:, -1]]

    inv_S_x = np.linalg.inv(S_x)
    inv_S_u = np.linalg.inv(S_u)

    for k in range(1, params['n']):
        constr += [0 == inv_S_x @ (x_unscaled[k] - x_bar[:, k] - dx[:, k])]
        constr += [0 == inv_S_u @ (u_unscaled[k - 1] - u_bar[:, k - 1] - du[:, k - 1])]

        cost += w_tr * cp.sum_squares(scipy.linalg.block_diag(np.linalg.inv(S_x), np.linalg.inv(S_u)) @ cp.hstack((dx[:, k], du[:, k - 1])))
        cost += params['w_fuel'] * cp.norm(np.linalg.inv(S_u) @ u_unscaled[k - 1])

        for i in range(params['n_obs']):
            cost += params['w_o'] * cp.pos(g_bar_obs[i][k - 1] + grad_g_bar_obs[i][:, k - 1].T @ dx[:3, k - 1])

        constr += [x_unscaled[k] == cp.reshape(A_prop[:, k - 1], (params['n_states'], params['n_states'])) @ x_unscaled[k - 1]
                   + cp.reshape(B_prop[:, k - 1], (params['n_states'], params['n_controls'])) @ u_unscaled[k - 1]
                   + cp.reshape(C_prop[:, k - 1], (params['n_states'], params['n_controls'])) @ u_unscaled[k]
                   + z_prop[:, k - 1] + nu[:, k - 1]]

        cost += params['w_vc'] * cp.sum(cp.abs(nu[:params['n_states'] - params['n_obs'], k - 1]))

        constr += [x_unscaled[k] <= params['max_state'], x_unscaled[k] >= params['min_state']]
        constr += [u_unscaled[k - 1] <= params['max_control'], u_unscaled[k - 1] >= params['min_control']]

    # Add terminal cost for the final state (only the drone state, not obstacle states)
    cost += w_terminal * cp.sum_squares(x_unscaled[-1][:params['n_states'] - params['n_obs']] - params['final_state'])

    constr += [u_unscaled[-1] <= params['max_control'], u_unscaled[-1] >= params['min_control']]

    prob = cp.Problem(cp.Minimize(cost), constr)
    return prob



def straight_line_init(x, u, x_f, n, params):
    u_traj = np.repeat(np.expand_dims(u, axis=1), n, axis=1)

    x_traj = np.zeros((params['n_states'], n))
    x_traj[params['n_states'] - params['n_obs']:] = 0.0
    x_traj[6, :] = 1
    for k in range(n - 1):
        x_traj[:6, k] = x[:6] + (k / n) * (x_f[:6] - x[:6])
    x_traj[:6, -1] = x_f[:6]
    return x_traj, u_traj

def subproblem(x_bar, u_bar, A, B, obstacles, prob, params):
    prob.param_dict['x_bar'].value = x_bar
    prob.param_dict['u_bar'].value = u_bar

    A_bar, B_bar, B_minus_bar, z_bar = calculate_discretization(x_bar, u_bar, A, B, obstacles, params)
    prob.param_dict['A_d'].value = A_bar
    prob.param_dict['B_d'].value = B_bar
    prob.param_dict['B_minus_d'].value = B_minus_bar
    prob.param_dict['z_d'].value = z_bar

    for i, obs in enumerate(obstacles):
        prob.param_dict['g_bar_obs_' + str(i)].value = g_bar_obs(obs, x_bar[:3, 1:])
        prob.param_dict['grad_g_bar_obs_' + str(i)].value = grad_g_bar_obs(obs, x_bar[:3, 1:])

    prob.param_dict['w_tr'].value = params['w_tr']
    prob.param_dict['w_terminal'].value = 0 # Set the value for the terminal cost weight here

    prob.solve(solver=cp.ECOS, enforce_dpp=False, warm_start=True)

    x = params['S_x'] @ prob.var_dict['x'].value + np.expand_dims(params['c_x'], axis=1)
    u = params['S_u'] @ prob.var_dict['u'].value + np.expand_dims(params['c_u'], axis=1)

    return x, u


def main(params, wind_vector=None):
    # Initialize the optimization problem and analytical Jacobians
    prob = controller(params)
    A, B = analytical_jacobians(params)

    # Generate initial straight-line trajectory and control guesses
    x_bar, u_bar = straight_line_init(params['initial_state'], params['initial_control'], params['final_state'], params['n'], params)



    # Prepare the obstacles based on the parameters
    obstacles = []
    for i in range(params['n_obs']):
        obstacle = {'center': params['obstacle_centers'][i], 'radius': params['obstacle_radii'][i]}
        obstacles.append(obstacle)

    # Iteration counter
    k = 0

    print("Iteration Number")
    print("-----------------")

    while True:
        # Solve the subproblem and get updated trajectories and costs
        x, u = subproblem(x_bar, u_bar, A, B, obstacles, prob, params)

        # Update the trajectory estimates
        x_bar = x
        u_bar = u

        # Print the current iteration's results
        print("{:4d}".format(k))

        # Increment the tracking weight slightly to tighten the tracking requirement
        #params['w_tr'] = params['w_tr'] * 1.1

        # Increment the iteration counter
        k += 1

        # Stop the loop if the maximum number of iterations is reached
        if k > params['k_max']:
            break

    # Simulate the nonlinear system using the final trajectories and controls
    x_full = simulate_nonlinear_with_wind(x[:-params['n_obs'], 0], u, params, params['dt_sim'], wind_vector)

    print("Dimension of x_full:", x_full.shape)

    # Package the results into a dictionary
    result = dict(
        times=np.arange(0, params['total_time'], params['dt_sim']),
        drone_state=x_full,
        drone_positions=x_full[:, :3],
        drone_attitudes=x_full[:, 6:10],
        drone_forces=u[:, :3],
        drone_controls=u,
        obstacles=obstacles,
    )

    return result

def nonlinear_min_fuel_cost(result, type, params):
    u_traj = result["drone_controls"]

    fuel_cost = np.sum(np.linalg.norm(u_traj, axis=1) * params['dt_ss'])
    print(f"Total fuel cost for {type} is {fuel_cost}")

    violation_node = 0
    for obs in result['obstacles']:
        violation_node += np.sum(np.maximum(np.zeros_like(g_bar_obs(obs, result['drone_positions'].T)), g_bar_obs(obs, result['drone_positions'].T)))

    print(f"Total Node Violation: {np.sum(violation_node)}")

def plot_static_3d_with_nodes_and_count(result, wind, params, wind_vector):
    # Set the title based on wind condition
    title = 'Trajectory of Quadrotor with Wind' if wind else 'Trajectory of Quadrotor without Wind'

    drone_positions = result["drone_positions"]
    obstacles = result["obstacles"]

    fig = go.Figure()

    # Plot the drone trajectory
    fig.add_trace(go.Scatter3d(
        x=drone_positions[:, 0],
        y=drone_positions[:, 1],
        z=drone_positions[:, 2],
        mode='lines',
        line=dict(color='green', width=5),
        name='Drone Trajectory'
    ))

    # Add start and end points with separate labels
    fig.add_trace(go.Scatter3d(
        x=[drone_positions[0, 0]],
        y=[drone_positions[0, 1]],
        z=[drone_positions[0, 2]],
        mode='markers',
        marker=dict(size=10, color='blue', symbol='circle'),
        name='Start Point'
    ))

    fig.add_trace(go.Scatter3d(
        x=[drone_positions[-1, 0]],
        y=[drone_positions[-1, 1]],
        z=[drone_positions[-1, 2]],
        mode='markers',
        marker=dict(size=10, color='red', symbol='circle'),
        name='End Point'
    ))

    # Plot the obstacles as spheres
    for obs in obstacles:
        n = 20
        u = np.linspace(0, 2 * np.pi, n)
        v = np.linspace(0, np.pi, n)

        x = np.outer(np.cos(u), np.sin(v))
        y = np.outer(np.sin(u), np.sin(v))
        z = np.outer(np.ones(np.size(u)), np.cos(v))

        x = obs['radius'] * x
        y = obs['radius'] * y
        z = obs['radius'] * z

        points = np.array([x.flatten(), y.flatten(), z.flatten()])
        points = points.T + obs['center']

        fig.add_trace(go.Surface(
            x=points[:, 0].reshape(n, n),
            y=points[:, 1].reshape(n, n),
            z=points[:, 2].reshape(n, n),
            opacity=0.5,
            showscale=False,
            name='Obstacle'
        ))

    # Add node points at dt_ss intervals using params dictionary
    node_indices = list(range(0, len(drone_positions), int(params['dt_ss'] / params['dt_sim'])))
    node_positions = drone_positions[node_indices]

    # Print the total number of nodes
    print(f"Total number of nodes: {len(node_positions)}")

    fig.add_trace(go.Scatter3d(
        x=node_positions[:, 0],
        y=node_positions[:, 1],
        z=node_positions[:, 2],
        mode='markers',
        marker=dict(size=6, color='yellow'),
        name='Node Points'
    ))

    # Set the layout and axis labels, ensuring axes span from -20 to 20
    fig.update_layout(
        title=title,
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z',
            xaxis=dict(range=[-20, 20], showspikes=False),
            yaxis=dict(range=[-20, 20], showspikes=False),
            zaxis=dict(range=[-20, 20], showspikes=False),
            aspectmode='manual',
            aspectratio=dict(x=1, y=1, z=1),
        ),
        height=800,
        showlegend=True
    )

    fig.show()



def run_quadrotor_simulation_with_wind(wind=False):  # Change wind condition here
    # Initialize the parameters
    params = parameters()

    # Define the wind vector based on the wind parameter
    if wind:
        wind_vector = np.array([0.0, 0.10, 0.0])  # Constant Wind is present
    else:
        wind_vector = np.array([0.0, 0.0, 0.0])  # No wind

    # Run the main PTR function to get the result with or without wind
    result = main(params, wind_vector=wind_vector)

    # Plot the 3D static plot with node points and print the number of nodes
    plot_static_3d_with_nodes_and_count(result, wind, params, wind_vector)

    # Print the total fuel cost and node violation
    nonlinear_min_fuel_cost(result, "Quadrotor Simulation with Wind" if wind else "Quadrotor Simulation without Wind",
                            params)


# Call the function to run the simulation with or without wind
run_quadrotor_simulation_with_wind()



Iteration Number
-----------------
   0
   1
   2
   3
   4
   5
   6
   7
   8
   9
  10
  11
  12
  13
  14
  15
  16
  17
  18
  19
  20
Dimension of x_full: (305, 13)
Total number of nodes: 20


Total fuel cost for Quadrotor Simulation without Wind is 22.133451420968235
Total Node Violation: 0.012052303050904434


**Wind added to algorithm**

In [4]:
def run_quadrotor_simulation_with_wind(wind=True):  # Change wind condition here
    # Initialize the parameters
    params = parameters()

    # Define the wind vector based on the wind parameter
    if wind:
        wind_vector = np.array([0.0, 0.10, 0.0])  # Constant Wind is present
    else:
        wind_vector = np.array([0.0, 0.0, 0.0])  # No wind

    # Run the main PTR function to get the result with or without wind
    result = main(params, wind_vector=wind_vector)

    # Plot the 3D static plot with node points and print the number of nodes
    plot_static_3d_with_nodes_and_count(result, wind, params, wind_vector)

    # Print the total fuel cost and node violation
    nonlinear_min_fuel_cost(result, "Quadrotor Simulation with Wind" if wind else "Quadrotor Simulation without Wind",
                            params)


# Call the function to run the simulation with or without wind
run_quadrotor_simulation_with_wind()

Iteration Number
-----------------
   0
   1
   2
   3
   4
   5
   6
   7
   8
   9
  10
  11
  12
  13
  14
  15
  16
  17
  18
  19
  20
Dimension of x_full: (305, 13)
Total number of nodes: 20


Total fuel cost for Quadrotor Simulation with Wind is 22.133451420968235
Total Node Violation: 41.71118972751173


**MPC style SCP algorithm**

In [ ]:
def run_quadrotor_simulation_with_wind():
    # Initialize the parameters
    params = parameters()

    # Define the wind vector for the simulation
    wind_vector = np.array([0.0, 0.0, 0.0])  # Wind is present

    # Run the MPC-style SCP over the trajectory
    for node in range(int(params['total_time'] / params['dt_ss'])):
        print(f"Running SCP for Node {node}")

        # Reset final state to the desired state
        params['final_state'] = np.array([10, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0])

        # Generate initial straight-line trajectory and control guesses for the current node
        x_bar, u_bar = straight_line_init(params['initial_state'], params['initial_control'], params['final_state'], params['n'], params)

        # Run the main function to get the result for the current node
        result = main(params, wind_vector=wind_vector)

        # Print the total time for the current iteration
        print(f"Total time remaining for Node {node}: {params['total_time']} seconds")

        # Plot the 3D static plot with node points and print the number of nodes
        plot_static_3d_with_nodes_and_count(result, True, params, wind_vector)

        # Print the total fuel cost and node violation
        nonlinear_min_fuel_cost(result, f"Quadrotor Simulation at Node {node} with Wind", params)

        # Update initial state for the next node using the second node's state from the current result
        initial_state = result['drone_state'][(int(params['dt_ss'] / params['dt_sim']))]

        # Update parameters for the next iteration
        params['initial_state'] = initial_state
        params['total_time'] -= params['dt_ss']  # Reduce total time by the discretization step

# Call the function to run the simulation with wind
run_quadrotor_simulation_with_wind()